In [ ]:
!pip install shap
!pip install xgboost

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/connect2robiul/CSVfile/master/GGS_new.csv",sep=";")
df.head()

In [ ]:
df["relig"] = df["relig"]+1
df["relig_type"] = df["relig_type"]+1
df["relig_fre"] = df["relig_fre"]+1
df[["relig","relig_type","relig_fre"]] = df[["relig","relig_type","relig_fre"]].fillna(0)
df["relig"] = df["relig"].astype(int)
df["relig_type"] = df["relig_type"].astype(int)
df["relig_fre"] = df["relig_fre"].astype(int)
df = df.fillna("Data_is_missing")
df["gen_binary"] = df["generations"].map({1:1,2:1,3:1,4:1,5:2,6:2})  # 1 is Soviet and 2 is Non-Soviet
df = df.drop(["ID","sg","sge","sgl","age","generations2","generations"],axis=1)
df.head()

In [ ]:
object_col = df.select_dtypes(exclude='object').columns.tolist()

In [ ]:
from sklearn.preprocessing import LabelEncoder
df_new = df.select_dtypes(include='object').apply(LabelEncoder().fit_transform)
df_new.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
df_new = df.select_dtypes(include='object').apply(LabelEncoder().fit_transform)
df_new.head()

In [ ]:
oject_col =df.select_dtypes(include='object').columns.tolist()
for col in oject_col:
    df[col] = df_new[col]
df.head()

In [ ]:
df_xgboost = df.copy()

In [ ]:
LEARNING_RATE = [0.001,0.01,0.1,.3,.5,1]
EVAL_METRIC = ['auc']
SEED = 0
CV = 5
MAX_DEPTH = [2,3,4,5,6]
ITTERATION = [5000]
EARLY_STOP = [50]
VERBOSE = 20

In [ ]:
from sklearn.model_selection import train_test_split 
import xgboost as xgb
df_new = df_xgboost.copy()
col = "gen_binary"
y = df_new[col]
X = df_new.drop(col,axis=1)
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

clf_xgboost = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed= SEED 
)
param_xgboost = {
    'max_depth': MAX_DEPTH,
    'n_estimators': ITTERATION,
    'learning_rate': LEARNING_RATE ,
    'eval_metric' : EVAL_METRIC ,
    'early_stopping_rounds' : EARLY_STOP
}
GridSearch_Xgboost = GridSearchCV(
    estimator=clf_xgboost,
    param_grid=param_xgboost,
    cv = CV,
    verbose=VERBOSE
)

results_Xgboost = GridSearch_Xgboost.fit(X_train,y_train)

In [ ]:
results_Xgboost.best_params_

In [ ]:
results_Xgboost.best_estimator_

In [ ]:
y_pred = results_Xgboost.best_estimator_.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
print(accuracy_score(y_test, y_pred) , "accuracy_score")
print(precision_score(y_test, y_pred) , 'precision_score')
print(recall_score(y_test, y_pred), "printrecall_score")
print(f1_score(y_test, y_pred) , "f1_score")

In [ ]:
import shap
shap.initjs()
explainer = shap.TreeExplainer(results_Xgboost.best_estimator_)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values, X)